In [ ]:
import os
os.chdir('..')
os.chdir('..')
import scvelo as scv
import scanpy as sc
import cell2fate as c2f
import pickle as pickle
from eval_utils import cross_boundary_correctness
from datetime import datetime
import pandas as pd
import numpy as np
from os.path import exists
import matplotlib.pyplot as plt
import torch
import unitvelo as utv
method = 'Cell2fateDynamicalModelLambda1_500epochs_10samples'
data_dir = '/nfs/team283/aa16/data/fate_benchmarking/benchmarking_datasets/'
save_dir = '/nfs/team283/aa16/data/fate_benchmarking/benchmarking_results/'
datasets = ['Pancreas_with_cc',  'DentateGyrus' , 'MouseBoneMarrow', 'MouseErythroid', 'HumanBoneMarrow']
n_genes_list = np.array((2000, 3000))
n_counts_list = np.array((10, 20))

for i in range(len(n_genes_list)):
    for j in range(len(n_counts_list)):
        for k in (3,0,1,2,4):
            print(i)
            print(j)
            print(k)
            dataset = datasets[k]
            n_genes = n_genes_list[i]
            min_counts = n_counts_list[j]
            model_index = str(i) + '-' + str(j) + '-' + str(k)
            save_name = method + '_'
            if exists(save_dir + save_name + '_CBDC_fullBenchmark.csv'):
                tab = pd.read_csv(save_dir + save_name + '_CBDC_fullBenchmark.csv', index_col = 0)
                if model_index in tab.index:
                    continue
            adata = sc.read_h5ad(data_dir + dataset + '/' + dataset + '_anndata.h5ad')
            adata = c2f.utils.get_training_data(adata, cells_per_cluster = 10**6, cluster_column = 'clusters',
                                            remove_clusters = [], min_shared_counts = min_counts, n_var_genes= n_genes)
            c2f.Cell2fate_DynamicalModel.setup_anndata(adata, spliced_label='spliced', unspliced_label='unspliced')    
            n_modules = c2f.utils.get_max_modules(adata)
            mod = c2f.Cell2fate_DynamicalModel(adata,
                                               n_modules = n_modules,
                                               activation_rate_hyp_prior={"mean_hyp_prior_mean": 1., "mean_hyp_prior_sd": 0.33,
                                               "sd_hyp_prior_mean": 0.33, "sd_hyp_prior_sd": 0.1},)   
            mod.train(max_epochs = 500)
            sample_kwarg = {"num_samples": 10, "batch_size" : adata.n_obs,
                 "use_gpu" : True, 'return_samples': False}
            adata = mod.export_posterior(adata, sample_kwargs = sample_kwarg)
            mod.compute_and_plot_total_velocity_scvelo(adata, save = False, delete = False)
            # Calculate performance metrics:
            file = open(data_dir + dataset + '/' + dataset + '_groundTruth.pickle' ,'rb')
            ground_truth = pickle.load(file)
            metrics = utv.evaluate(adata, ground_truth, 'clusters', 'velocity')
            cb_score = [np.mean(metrics['Cross-Boundary Direction Correctness (A->B)'][x])
                        for x in metrics['Cross-Boundary Direction Correctness (A->B)'].keys()]
            if exists(save_dir + save_name + '_CBDC_fullBenchmark.csv'):
                tab = pd.read_csv(save_dir + save_name + '_CBDC_fullBenchmark.csv', index_col = 0)
            else:
                c_names = ['CBDC']
                tab = pd.DataFrame(columns = c_names)
            tab.loc[model_index, 'CBDC'] = np.mean(cb_score)
            tab.to_csv(save_dir + save_name + '_CBDC_fullBenchmark.csv')
tab = pd.read_csv(save_dir + save_name + '_CBDC_fullBenchmark.csv', index_col = 0)
tab.loc['AVERAGE', 'CBDC'] = np.mean(tab['CBDC'])
tab.to_csv(save_dir + save_name + '_CBDC_fullBenchmark.csv')

Global seed set to 0


(Running UniTVelo 0.2.5)
2023-10-29 18:22:54
0
0
3
Keeping at most 1000000 cells per cluster
Filtered out 46616 genes that are detected 10 counts (shared).
Extracted 2000 highly variable genes.
Leiden clustering ...
Number of Leiden Clusters: 9
Maximal Number of Modules: 10


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Sampling global variables, sample: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:02<00:00,  3.45it/s]
Computing total RNAvelocity ...


  0%|          | 0/9815 [00:00<?, ?cells/s]

...

Number of Leiden Clusters: 8
Maximal Number of Modules: 9
Sampling global variables, sample: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:01<00:00,  6.95it/s]
Computing total RNAvelocity ...


  0%|          | 0/2600 [00:00<?, ?cells/s]

# Cross-Boundary Direction Correctness (A->B)
{('dividing', 'progenitors'): 0.520749845746637, ('progenitors', 'activating'): 0.7886209025044356}
Total Mean: 0.6546853741255363
# In-cluster Coherence
{'B cell lineage': 0.6741054, 'activating': 0.8011247, 'dividing': 0.9028957, 'macrophages': 0.5683921, 'progenitors': 0.8539143}
Total Mean: 0.7600864171981812
0
0
4
Keeping at most 1000000 cells per cluster
Filtered out 6962 genes that are detected 10 counts (shared).
Extracted 2000 highly variable genes.
Leiden clustering ...
         Falling back to preprocessing with `sc.pp.pca` and default params.


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Number of Leiden Clusters: 10
Maximal Number of Modules: 11
Epoch 297/500:  59%|███████████████████████████████████████████████████████████████████████████████████████████▏                                                              | 296/500 [08:44<03:39,  1.08s/it, v_num=1, elbo_train=9.83e+6]